# **Modeling Sale Price using Regression**

Part of CRISP-DM **Modelling** and **Evaluation**

## Objectives

* Fit and evaluate a regression model to predict sale prices of inherited houses
## Inputs

* outputs/datasets/collection/house_prices.csv
* Instructions on which variables to use for data cleaning and feature engineering, found in the respective notebooks.

## Outputs

* Train set (features and target)
* Test set (features and target)
* ML pipeline to predict house prices
* Feature Importance Plot


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("outputs/datasets/collection/house_prices.csv") 

print(df.shape)
df.head(3)

---

# ML Pipeline: Regression

### Create the ML Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

# Data Cleaning
from feature_engine.selection import DropFeatures
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import ArbitraryNumberImputer
from feature_engine.imputation import CategoricalImputer

# Feature Engineering
from feature_engine import creation
from feature_engine.encoding import OrdinalEncoder
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer
from feature_engine.selection import SmartCorrelatedSelection

# Feature Scaling
from sklearn.preprocessing import StandardScaler

# Feature Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms 
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

# Pipeline optimization
def PipelineOptimization(model):
    pipeline_base = Pipeline([

        # Data Cleaning - see Data Cleaning Notebook
        ( 'drop',  DropFeatures(features_to_drop=['EnclosedPorch',
                                                  'WoodDeckSF']) ),
        ( 'mean',  MeanMedianImputer(imputation_method='mean',
                                     variables=['LotFrontage',
                                                'BedroomAbvGr']) ),
        ( 'median',  MeanMedianImputer(imputation_method='median',
                                     variables=['GarageYrBlt']) ),
        ( 'arbitrary', ArbitraryNumberImputer(arbitrary_number=0,
                                     variables=['2ndFlrSF',
                                                'MasVnrArea']) ),
        ( 'categorical',  CategoricalImputer(imputation_method='missing',
                                     fill_value='Unf',
                                     variables=['GarageFinish',
                                                'BsmtFinType1']) ),

        # Feature Engineering - see Feature Engineering Notebook
        ('ordinal_encoder', OrdinalEncoder(encoding_method='arbitrary',
                                            variables = ['BsmtExposure',
                                                         'BsmtFinType1',
                                                         'GarageFinish',
                                                         'KitchenQual'])),

        ('log_transformer', vt.LogTransformer(variables=['1stFlrSF',
                                                         'GrLivArea',
                                                         'LotArea',
                                                         'LotFrontage'],
                                                         base='e')),

        ('power_transformer', vt.PowerTransformer(variables=['BsmtUnfSF',
                                                             'MasVnrArea',
                                                             'OpenPorchSF',
                                                             'TotalBsmtSF'])),

        ('yeo_johnson_transformer', vt.YeoJohnsonTransformer(variables=['GarageArea'])),

        ('windsorizer', Winsorizer(capping_method='iqr', tail='both', fold=1.5,
                                    variables = ['GrLivArea',
                                                 'OpenPorchSF',
                                                 'TotalBsmtSF'])),

        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None,
            method="spearman", threshold=0.6, selection_method="variance")),
        
        # Feature Scaling
        ("scaler", StandardScaler()),

        # Feature Selection
        ("feat_selection", SelectFromModel(model)),

        # ML Algorithms
        ("model", model),
    ])

    return pipeline_base

Custom Class for hyperparamter optimization

In [ ]:
# Code from walkthrough project 02, modeling and evaluation notebook

from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

## Split Train Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0,
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

## Grid Search CV - Sklearn

### Use default hyperparameters to find most suitable algorithm

In [ ]:
# Code from walkthrough project 02, modeling and evaluation notebook

models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

We do a hyperparameter optimization search using default hyperparameters

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

We check the results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

### We do an extensive search on the most suitable algorithm to find the best hyperparameter configuration.

We define model and parameters, for Extensive Search

In [ ]:
# defining model parameters for a more extensive search

models_search = {
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
}

# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
# In a workplace project, you may consider more hyperparameters and spend more time in this step
# https://inria.github.io/scikit-learn-mooc/python_scripts/ensemble_hyperparameters.html

params_search = {
    "GradientBoostingRegressor": {
          'model__n_estimators': [50, 75, 100], # gives little improvements we stick to default = 100
          'model__max_depth': [3, 6, 9], # gives little improvement we stick to default = 3
          'model__learning_rate': [0.05, 0.1, 0.2], # This learning rate consistently gave the best r2 scores, default = 0.1
          # 'model__learning_rate': [0.05, 0.07, 0.1], # default = 0.1
          'model__min_samples_split': [2, 4, 8, 75], # setting this to 75 also gives very good results, but we will stick to the default = 2
          'model__min_samples_leaf': [1, 2, 8, 75], # changing this parameter also seems to cause slight overfitting - keep default = 1
          'model__max_leaf_nodes': [None, 25, 50, 75], # Changing this parameter seems to often cause slight overfitting - keep default = None
    }
}

Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

We check the results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

We get the best model name programmatically

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

We get the parameters for the best model

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

We define the best regressor pipeline

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

## Assess feature importance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

data_cleaning_feat_eng_steps = 11 # how many data cleaning and feature engineering does your pipeline have?
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps]).transform(X_train).columns)

best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
          'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],
          'Importance': best_regressor_pipeline['model'].feature_importances_})
  .sort_values(by='Importance', ascending=False)
  )

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

## Evaluate on Train and Test Sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error 
import numpy as np

def regression_performance(X_train, y_train, X_test, y_test,pipeline):
	print("Model Evaluation \n")
	print("* Train Set")
	regression_evaluation(X_train,y_train,pipeline)
	print("* Test Set")
	regression_evaluation(X_test,y_test,pipeline)

def regression_evaluation(X,y,pipeline):
  prediction = pipeline.predict(X)
  print('R2 Score:', r2_score(y, prediction).round(3))  
  print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))  
  print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))  
  print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, prediction)).round(3))
  print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test,pipeline, alpha_scatter=0.5):
  pred_train = pipeline.predict(X_train)
  pred_test = pipeline.predict(X_test)


  fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
  sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
  sns.lineplot(x=y_train , y=y_train, color='red', ax=axes[0])
  axes[0].set_xlabel("Actual")
  axes[0].set_ylabel("Predictions")
  axes[0].set_title("Train Set")

  sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
  sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
  axes[1].set_xlabel("Actual")
  axes[1].set_ylabel("Predictions")
  axes[1].set_title("Test Set")
  plt.savefig(f'docs/plots/regression_performance.png', bbox_inches='tight')  
  plt.show()

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

## Refit pipeline with best features

### Rewrite Pipeline

We leave only the steps that affect the most important features identified by our analysis of the model, these are:
'OverallQual', 'TotalBsmtSF', '2ndFlrSF', and 'GarageArea'.

In [ ]:
def PipelineOptimization(model):
    pipeline_base = Pipeline(steps=[

        # Data Cleaning - see Data Cleaning Notebook
        # ( 'drop',  DropFeatures(features_to_drop=['EnclosedPorch',
        #                                           'WoodDeckSF']) ),
        # ( 'mean',  MeanMedianImputer(imputation_method='mean',
        #                              variables=['LotFrontage',
        #                                         'BedroomAbvGr']) ),
        # ( 'median',  MeanMedianImputer(imputation_method='median',
        #                              variables=['GarageYrBlt']) ),
        ( 'arbitrary', ArbitraryNumberImputer(arbitrary_number=0,
                                     variables=['2ndFlrSF',
        #                                        'MasVnrArea'
                                                ]) ),
        # ( 'categorical',  CategoricalImputer(imputation_method='missing',
        #                              fill_value='Unf',
        #                              variables=['GarageFinish',
        #                                         'BsmtFinType1']) ),

        # Feature Engineering - see Feature Engineering Notebook
        # ('ordinal_encoder', OrdinalEncoder(encoding_method='arbitrary',
        #                                     variables = ['BsmtExposure',
        #                                                  'BsmtFinType1',
        #                                                  'GarageFinish',
        #                                                  'KitchenQual'])),

        # ('log_transformer', vt.LogTransformer(variables=['1stFlrSF',
        #                                                  'GrLivArea',
        #                                                  'LotArea',
        #                                                  'LotFrontage'],
        #                                                  base='e')),

        ('power_transformer', vt.PowerTransformer(variables=[
        #                                                    'BsmtUnfSF',
        #                                                    'MasVnrArea',
        #                                                    'OpenPorchSF',
                                                             'TotalBsmtSF'])),

        ('yeo_johnson_transformer', vt.YeoJohnsonTransformer(variables=['GarageArea'])),

        ('windsorizer', Winsorizer(capping_method='iqr', tail='both', fold=1.5,
                                    variables = [
        #                                        'GrLivArea',
        #                                        'OpenPorchSF',
                                                 'TotalBsmtSF'])),

        # ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None,
        #     method="spearman", threshold=0.6, selection_method="variance")),
        
        # Feature Scaling
        ("scaler", StandardScaler()),

        # Feature Selection
        # ("feat_selection", SelectFromModel(model)),

        # ML Algorithms
        ("model", model),
    ])

    return pipeline_base

### Split Train Test Set, considering only with best features

In [ ]:
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",  X_test.shape, y_test.shape)
X_train.head(3)

We are using the same model from the last GridCV search

In [ ]:
models_search # GradientBoostingRegressor

And the best parameters from the last GridCV search

In [ ]:
best_parameters

In [ ]:
params_search = {
    "GradientBoostingRegressor": {
        'model__n_estimators': [50],
        'model__max_depth': [3],
        'model__learning_rate': [0.1],
        'model__min_samples_split': [75],
        'model__min_samples_leaf': [1],
        'model__max_leaf_nodes': [25],
    }
}

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

In [ ]:
# Defining the best pipeline

best_pipeline_regression = grid_search_pipelines[best_model].best_estimator_
best_pipeline_regression

---

# Push files to Repo

The following files will be created and pushed to the repo:

* Train Set
* Test Set
* Modeling Pipeline
* Feature importance Plot

In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/predict_sale_price/{version}'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)
    

### Train Set: Features and Target

In [ ]:
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train.head()

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

### Test Set: Features and Target

In [ ]:
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test.head()

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

### Modelling Pipeline

In [ ]:

best_pipeline_regression

In [ ]:
joblib.dump(value=best_pipeline_regression, filename=f"{file_path}/regression_pipeline.pkl")

### Feature Importance Plot

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')
## Save to docs folder for documentation
plt.savefig(f'docs/plots/features_importance.png', bbox_inches='tight') 